### We stream a portion of documents from Frontier indexed data and testing ways to parse SQL queries.

In [1]:
%matplotlib inline
import datetime
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from elasticsearch import Elasticsearch, helpers
es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

### will look up only data from the current month

In [2]:
today = datetime.date.today()

ind='frontier-new-'+str(today.year)+'-'+str(today.month).zfill(2)
print(ind)

frontier-new-2018-03


### Streaming data

In [3]:
my_query={
    "size": 0,
    "_source": ["sqlquery"],
  "query": {
    "bool": {
      "must": [
        {
          "term": {"tableowner": ""},
          "term": {"taskid":13245563}
#           "term": {"computingsite":"BOINC-ES"}
        }
      ]
    }
  }
}

res = helpers.scan(es, query=my_query, index=ind, scroll='5m', timeout="5m", size=1000)

limit = 10
for r in res:
    print('===========================================')
    limit-=1
    if not limit: break
    parsed={'from':'', 'order':'', 'where':''}
    
    q=r['_source']['sqlquery']
#     print(q)
    print('----------------- cleaned up ---------------')

    if q.startswith('SELECT '): q=q[7:]

    if ' ORDER BY' in q:
        parsed['order']=q[q.index(' ORDER BY'):]
        q=q[:q.index(' ORDER BY')]
#     print(q)
    
    print('----------------- parse WHERE --------------')
    if ' WHERE ' in q:
        where = q[q.index(' WHERE ') + 7:]
        q=q[:q.index(' WHERE ')]
    else: where=''
    print(q)
    print('WHERE: ', where)
    

----------------- cleaned up ---------------
----------------- parse WHERE --------------
/*+ NO_BIND_AWARE QB_NAME(MAIN) INDEX_RS_ASC(@MAIN COOL_I3@MAIN (USER_TAG_ID NEW_HEAD_ID CHANNEL_ID IOV_SINCE IOV_UNTIL)) INDEX_RS_ASC(@MAIN COOL_P5@MAIN (PAYLOAD_ID)) LEADING(@MAIN COOL_C2@MAIN COOL_I3@MAIN COOL_P5@MAIN) USE_NL(@MAIN COOL_I3@MAIN) USE_NL(@MAIN COOL_P5@MAIN) INDEX(@MAX1 COOL_I1@MAX1 (USER_TAG_ID NEW_HEAD_ID CHANNEL_ID IOV_SINCE IOV_UNTIL)) */ COOL_I3.OBJECT_ID AS "OBJECT_ID", COOL_I3.CHANNEL_ID AS "CHANNEL_ID", COOL_I3.IOV_SINCE AS "IOV_SINCE", COOL_I3.IOV_UNTIL AS "IOV_UNTIL", COOL_I3.USER_TAG_ID AS "USER_TAG_ID", COOL_I3.SYS_INSTIME AS "SYS_INSTIME", COOL_I3.LASTMOD_DATE AS "LASTMOD_DATE", COOL_I3.ORIGINAL_ID AS "ORIGINAL_ID", COOL_I3.NEW_HEAD_ID AS "NEW_HEAD_ID", COOL_I3.PAYLOAD_ID AS "PAYLOAD_ID", COOL_P5."status" AS "status", COOL_P5."posX" AS "posX", COOL_P5."posY" AS "posY", COOL_P5."posZ" AS "posZ", COOL_P5."sigmaX" AS "sigmaX", COOL_P5."sigmaY" AS "sigmaY", COOL_P5."sigma